In [2]:
# IMPORTANT: Just press run all above. then quit.

In [3]:
# !jupyter-nbextension enable nglview --py --sys-prefix

In [4]:
# analysis notebook
# methods are cMD, aMD, GaMD.
# in snakemake rule, method needs to be passed as a parameter. s.t. it is accessible via: snakemake.params.method

import matplotlib

#%matplotlib inline
# matplotlib.use("Agg")

import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import pandas as pd

sys.path.append(os.getcwd())
import src
from src.noe import compute_NOE_mdtraj, plot_NOE
from src.pca import make_PCA, plot_PCA, getDih
from src.pyreweight import reweight
from src.analyse import getOmega
from src.utils import json_load
from sklearn.decomposition import PCA

from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

import nglview as nv

In [5]:
compound_index = int(snakemake.wildcards.compound_dir)
compound_index

In [6]:
# Load cluster data
clus_g = md.load(snakemake.input.clusters_0)
clus_a = md.load(snakemake.input.clusters_1)
clus_c = md.load(snakemake.input.clusters_2)

clus_a.superpose(clus_g)
clus_c.superpose(clus_g)


clusters = [clus_g, clus_a, clus_c]

In [7]:
# Load NOE data
NOE = src.noe.read_NOE(snakemake.input.noe)
NOE_number = 0

a1 = eval(NOE["Atom 1"][NOE_number])
a2 = eval(NOE["Atom 2"][NOE_number])

NOE_pairs = []

for a in a1:
    for b in a2:
        NOE_pairs.append([a, b])

In [8]:
%gui asyncio

In [9]:
shell = get_ipython()
kernel = shell.kernel

captured_events = []


def execute_request(stream, ident, parent):
    "Overwrite function to store the stream / ident /parent instead of calling kernel.execute_request"
    captured_events.append((stream, ident, parent))


def comm_msg(stream, ident, parent):
    "Overwrite function to add a logging (print) msg when comm_msg events come through"
    # print("in comm_msg")
    return kernel.comm_manager.comm_msg(stream, ident, parent)


def start_capturing():
    "Overwrite the kernel shell handlers"
    kernel.shell_handlers["execute_request"] = execute_request
    kernel.shell_handlers["comm_msg"] = comm_msg


def stop_capturing():
    "rever the kernel shell handler functions to their defaults"
    kernel.shell_handlers["execute_request"] = kernel.execute_request
    kernel.shell_handlers["comm_msg"] = kernel.comm_manager.comm_msg

In [10]:
import nglview as nv
from IPython.display import display
import time

data = []
views = []
view_frames = []
# def g():
for i in range(len(clusters)):
    v = nv.show_mdtraj(clusters[i])
    # v.clear()
    v.add_ball_and_stick()
    v.add_representation(
        "distance", atom_pair=NOE_pairs, label_color="black", color="red"
    )
    # add_distance([0,5])
    def func():
        image = v.render_image(trim=True)
        while not image.value:
            time.sleep(0.2)
        data.append(image.value)
        # print(len(data))
        # v.close()

    def on_display(self):
        # func()
        v._run_on_another_thread(func)
        # func()
        time.sleep(1)  # Might need to increase the timeout here.

    v.on_displayed(on_display)
    display(v)
    views.append(v)
    view_frames.append(clusters[i].n_frames)
    # v._run_on_another_thread(display(v))
    # await asyncio.sleep(2)

In [12]:
start_capturing()
shell.execution_count += 1
while True:
    # While this loop is running, all further 'execute_request' messages will get captured
    if len(data) == len(clusters):
        print("widget value changed: breaking from loop")
        break  # user changed the value
    kernel.do_one_iteration()  # same thing an eventloop like %gui asyncio would do
    time.sleep(0.01)
print(len(data))
stop_capturing()

# ### Once the widget value has changed, 'replay' the captured execute_request messages
# ### Unfortunately the output shows up in this cell, not in the cells where the
# ### original input code is at...
# for stream, ident, parent in captured_events:
#     kernel.execute_request(stream, ident, parent)
sys.stdout.flush()
sys.stderr.flush()
for stream, ident, parent in captured_events:
    kernel.set_parent(ident, parent)
    kernel.execute_request(stream, ident, parent)

In [13]:
# # Use this to sync views..
# def on_change(change):
#     views[1]._set_camera_orientation(change['new'])
#     views[2]._set_camera_orientation(change['new'])
#     #views[3]._set_camera_orientation(change['new'])

# views[0].observe(on_change, ['_camera_orientation'])

In [14]:
nv.write_html(snakemake.output.visual, views, frame_range=(4, 4, 1))
print("Done")

In [15]:
NOE_pairs